In [405]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from braindecode.models import EEGConformer
import pandas as pd
from collections import OrderedDict
from CKA_functions import adjacency_matrix_motion,adjacency_matrix_distance_motion
import math
import importlib
import RGNN
importlib.reload(RGNN)
from RGNN import ShallowRGNNNet
from torchinfo import summary

In [406]:
in_chans = 22
n_classes = 4
input_window_samples = 1125

adj_m,pos = adjacency_matrix_motion()
#print(adj_m)
adj_dis_m, dm = adjacency_matrix_distance_motion(pos,delta=5)
dm
torch_tensor = torch.from_numpy(dm)
edge_weight = torch_tensor.reshape(-1)
print(edge_weight.shape)
model = ShallowRGNNNet(in_chans,n_classes,input_window_samples,edge_weight)

torch.Size([484])


In [407]:
print(model)

ShallowRGNNNet(
  (temporal): Conv2d(1, 10, kernel_size=(1, 25), stride=(1, 1))
  (rgnn): SimpleGCNNet(
    (rgnn): SGConv(55, 5, K=1)
  )
  (batch_norm): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): AvgPool2d(kernel_size=(1, 20), stride=(1, 20), padding=0)
  (dropout): Dropout(p=0.7, inplace=False)
  (fc): Linear(in_features=1100, out_features=4, bias=True)
)


In [408]:

# Define a threshold distance below which nodes will be connected
threshold = 0  # Adjust as needed

source_nodes = []
target_nodes = []

# Iterate over all elements in the distance matrix, including self-loops and duplicates
for i in range(dm.shape[0]):
    for j in range(dm.shape[1]):  # Iterate over all pairs, including (i, i)
        if dm[i, j] >= threshold:  # If the distance meets the condition
            source_nodes.append(i)  # Source node
            target_nodes.append(j)  # Target node

# Create the edge_index tensor
edge_index = torch.tensor([source_nodes, target_nodes], dtype=torch.long)

print("Distance Matrix Shape:", dm.shape)
print("Edge Index:\n", edge_index.shape)

Distance Matrix Shape: (22, 22)
Edge Index:
 torch.Size([2, 484])


In [409]:
print(edge_index.shape)

torch.Size([2, 484])


In [410]:
summary(model,input_size=(2, 22, 1125),edge_index=edge_index, col_names=["input_size", "output_size", "num_params", "kernel_size"])

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Kernel Shape
ShallowRGNNNet                           [2, 22, 1125]             [2, 4]                    --                        --
├─Conv2d: 1-1                            [2, 1, 22, 1125]          [2, 10, 22, 1101]         260                       [1, 25]
├─BatchNorm2d: 1-2                       [2, 10, 22, 1101]         [2, 10, 22, 1101]         20                        --
├─AvgPool2d: 1-3                         [2, 10, 22, 1101]         [2, 10, 22, 55]           --                        [1, 20]
├─SimpleGCNNet: 1-4                      [2, 10, 22, 55]           [2, 10, 22, 5]            484                       --
│    └─SGConv: 2-1                       [2, 10, 22, 55]           [2, 10, 22, 5]            --                        --
│    │    └─SumAggregation: 3-1          [2, 10, 484, 55]          [2, 10, 22, 55]           --                        --
│   